# Edge Detection with CUDA and Numba
Demonstrates:
- 2D array operations
- Shared memory usage
- Grid/block concepts

In [ ]:
!pip install numba opencv-python
import numpy as np
from numba import cuda
import cv2
import matplotlib.pyplot as plt
from time import time

In [ ]:
SOBEL_X = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]], dtype=np.float32)

SOBEL_Y = np.array([[-1, -2, -1],
                    [ 0,  0,  0],
                    [ 1,  2,  1]], dtype=np.float32)

@cuda.jit
def edge_detect_shared(image, output):
    # Shared memory for 3x3 neighborhood
    BLOCK_SIZE = 16
    shared = cuda.shared.array((BLOCK_SIZE+2, BLOCK_SIZE+2), dtype=np.float32)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bx = cuda.blockIdx.x
    by = cuda.blockIdx.y
    
    # Global indices
    x = bx * BLOCK_SIZE + tx
    y = by * BLOCK_SIZE + ty
    
    # Load data into shared memory
    if x < image.shape[0] and y < image.shape[1]:
        shared[ty+1, tx+1] = image[x, y]
        
        # Load halo regions
        if tx == 0 and x > 0:
            shared[ty+1, 0] = image[x-1, y]
        if tx == BLOCK_SIZE-1 and x < image.shape[0]-1:
            shared[ty+1, tx+2] = image[x+1, y]
        if ty == 0 and y > 0:
            shared[0, tx+1] = image[x, y-1]
        if ty == BLOCK_SIZE-1 and y < image.shape[1]-1:
            shared[ty+2, tx+1] = image[x, y+1]
            
    cuda.syncthreads()
    
    if x < image.shape[0]-1 and y < image.shape[1]-1 and x > 0 and y > 0:
        gx = 0.0
        gy = 0.0
        
        # Sobel filter computation
        for i in range(3):
            for j in range(3):
                pixel = shared[ty+i, tx+j]
                gx += pixel * SOBEL_X[i, j]
                gy += pixel * SOBEL_Y[i, j]
                
        # Magnitude of gradient
        output[x, y] = (gx*gx + gy*gy)**0.5

In [ ]:
def process_image(image, use_gpu=True):
    if not use_gpu:
        gx = cv2.Sobel(image, cv2.CV_32F, 1, 0)
        gy = cv2.Sobel(image, cv2.CV_32F, 0, 1)
        return np.sqrt(gx*gx + gy*gy)
    
    output = np.zeros_like(image, dtype=np.float32)
    d_image = cuda.to_device(image)
    d_output = cuda.to_device(output)
    
    threadsperblock = (16, 16)
    blockspergrid_x = (image.shape[0] + threadsperblock[0] - 1) // threadsperblock[0]
    blockspergrid_y = (image.shape[1] + threadsperblock[1] - 1) // threadsperblock[1]
    blockspergrid = (blockspergrid_x, blockspergrid_y)
    
    edge_detect_shared[blockspergrid, threadsperblock](d_image, d_output)
    
    return d_output.copy_to_host()

In [ ]:
# Load and process test image
image = cv2.imread('your_image.jpg', cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255

# CPU timing
t0 = time()
cpu_result = process_image(image, use_gpu=False)
cpu_time = time() - t0

# GPU timing
t0 = time()
gpu_result = process_image(image, use_gpu=True)
gpu_time = time() - t0

# Display results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(image, cmap='gray')
axes[0].set_title('Original')
axes[1].imshow(cpu_result, cmap='gray')
axes[1].set_title(f'CPU Result ({cpu_time:.3f}s)')
axes[2].imshow(gpu_result, cmap='gray')
axes[2].set_title(f'GPU Result ({gpu_time:.3f}s)')
plt.show()